In [ ]:
import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU,Dense
from keras.layers import ZeroPadding2D,Flatten
from keras.layers import UpSampling2D,Lambda
from keras.layers import add, Concatenate,GlobalAveragePooling2D,Softmax
from keras.models import Model
import tensorflow as tf
from keras.applications import vgg19
import tensorflow_datasets as tfds
import os
import cv2 
from google.colab.patches import cv2_imshow
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [ ]:
!rm -r sample_data

In [ ]:
!curl --header 'Host: data.vision.ee.ethz.ch' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip' --output 'DIV2K_train_LR_bicubic_X2.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  882M  100  882M    0     0   156M      0  0:00:05  0:00:05 --:--:--  162M


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !zip -r /content/ZipLR.zip /content/lr/LR

  adding: content/lr/LR/ (stored 0%)


In [ ]:
! du -sh /content/drive/MyDrive/SRGAN/HR

du: cannot read directory '/content/drive/MyDrive/SRGAN/HR': Input/output error
4.0K	/content/drive/MyDrive/SRGAN/HR


In [ ]:
from google.colab import files
files.download("/content/drive/MyDrive/SRGAN/LRR")

In [ ]:
!unzip 'DIV2K_train_LR_bicubic_X2.zip' -d 'DATA'

In [ ]:
!curl --header 'Host: data.vision.ee.ethz.ch' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_unknown_X2.zip' --output 'DIV2K_train_LR_unknown_X2.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  700M  100  700M    0     0   136M      0  0:00:05  0:00:05 --:--:--  148M


In [ ]:
!unzip 'DIV2K_train_LR_unknown_X2.zip' -d 'DATA'

In [ ]:
!curl --header 'Host: data.vision.ee.ethz.ch' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:83.0) Gecko/20100101 Firefox/83.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --header 'Upgrade-Insecure-Requests: 1' 'https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip' --output 'DIV2K_train_HR.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3367M  100 3367M    0     0  44.0M      0  0:01:16  0:01:16 --:--:-- 41.1M


In [ ]:
!unzip 'DIV2K_train_HR.zip' -d 'train data'

In [ ]:
#Getting the path to all LR images having unknown downsampling
def getting_paths(dir_name,name):

# 
    Root=Path(dir_name)
    path= list(Root.glob('*.png'))

    values=[str(i+1) for i in range(len(path))]
    id=[]
    back_name='x2.png' if name=='LR' else '.png'

    for val in values:
       id.append((4-len(val))*'0'+val+back_name)
    path=[dir_name+i for i in id]
    return path



In [ ]:
LR_dir= 'DATA/DIV2K_train_LR_unknown/X2/'
HR_dir= 'train data/DIV2K_train_HR/'
path_to_LR= getting_paths(LR_dir,'LR')
print('The total LR image size',len(path_to_LR))
path_to_HR= getting_paths(HR_dir,'HR')
print('The toatl HR image size',len(path_to_HR))

The total LR image size 800
The toatl HR image size 800


### Image **augmentation**

In [ ]:
def augmentation(path_to_HR,HR_dir,path_to_LR,LR_dir):

    count=800 
    for i in range(800):

        if np.random.sample() >0.5:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])

            imgflip_HR= cv2.flip(img_HR,0) #flipping the image from x-axis
            imgflip_LR= cv2.flip(img_LR,0)
            #Getting the location of the images
           
            imgHR_loc= HR_dir+(4-len(str(count)))*'0' +str(count)+'.png'
            imgLR_loc= LR_dir+(4-len(str(count)))*'0' +str(count)+'x2.png'
    
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)


        if np.random.sample() >0.5:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])
            
            #Rotating the image at an angle of 90
            imgflip_HR= cv2.rotate(img_HR, cv2.ROTATE_90_CLOCKWISE)
            imgflip_LR= cv2.rotate(img_LR, cv2.ROTATE_90_CLOCKWISE)
            #Getting the location of the images

            imgHR_loc= HR_dir+(4-len(str(count)))*'0' +str(count)+'.png'
            imgLR_loc= LR_dir+(4-len(str(count)))*'0' +str(count)+'x2.png'
      
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)


        if np.random.sample() >0.5:

            count+=1
            img_HR= cv2.imread(path_to_HR[i])
            img_LR= cv2.imread(path_to_LR[i])
            
            #Rotating the image at an angle of 270
            imgflip_HR= cv2.rotate(img_HR, cv2.ROTATE_90_COUNTERCLOCKWISE) 
            imgflip_LR= cv2.rotate(img_LR, cv2.ROTATE_90_COUNTERCLOCKWISE) 
            
            #Getting the location of the images

            imgHR_loc= HR_dir+(4-len(str(count)))*'0' +str(count)+'.png'
            imgLR_loc= LR_dir+(4-len(str(count)))*'0' +str(count)+'x2.png'
       
            cv2.imwrite(imgHR_loc,imgflip_HR)
            cv2.imwrite(imgLR_loc,imgflip_LR)   




In [ ]:
augmentation(path_to_HR,HR_dir,path_to_LR,LR_dir)
path_to_LR= getting_paths(LR_dir,'LR')
print('The total LR image size after augmentation',len(path_to_LR))
path_to_HR= getting_paths(HR_dir,'HR')
print('The toatl HR image size after augmentation',len(path_to_HR))

The total LR image size after augmentation 1993
The toatl HR image size after augmentation 1993


In [ ]:
parent_dir = "/content/drive/MyDrive/SRGAN/5LR"
# directory=0
for a1 in range(35):
  # if a1%10 ==0:
    # directory +=1
  path = os.path.join(parent_dir, str(a1+1))
  os.mkdir(path)

In [ ]:
106726%5000

1726

In [ ]:
al = "he/haha/"
newp = al + str(4)+ "png"
print(newp)

he/haha/4png


In [ ]:
# @tf.function
def Total_patches(paths,patch_size,name):
    
    total_data=[]
    shapes_img=[]
    count=1
    for path in tqdm(paths):
         
        patch,sh,count=getting_patches(path,patch_size,name,count)
        # print(sh)
        total_data.extend(patch)
        shapes_img.extend(sh)
    loc='/content/drive/MyDrive/SRGAN/5LR'+'.pkl'   
    with open(loc,'wb') as f:
         pickle.dump(shapes_img,f)

#getting patches of the input images
def getting_patches(paths,patch_size,name,count):

    scale= 1 if name=='LR' else 2
    patch_size=patch_size*scale
    for i in range(1):
        patches=[]
        shapes=[]
        image= cv2.imread(paths, cv2.IMREAD_UNCHANGED)
        ny= image.shape[0]//patch_size
        nx= image.shape[1]//patch_size
        shapes.append([ny,nx])


        for i in range(ny):

          starty=i*patch_size

          for j in range(nx):

            start= j*patch_size
            end= patch_size + j*patch_size

            Lr_patch= image[starty:patch_size+starty,start:end]
            # Lr_patch= image[:patch_size,start:en
            # parent_dir="/content/drive/MyDrive/SRGAN/5LR/"
            # directory=1
            # if count%7500==0:
            #   directory +=1
            #   pathh = os.path.join(parent_dir, str(directory))
            #   os.mkdir(pathh)

            folder=1
            if count%7500 == 1:
              folder = + 1
            location='/content/drive/MyDrive/SRGAN/5LR/'+str(folder)+'/'+str(count)+'.png'
            # location= pathh +"/"+str(count)+'.png'

            count=count+1
            cv2.imwrite(location,Lr_patch)
            patches.append(Lr_patch)


    return patches,shapes,count


#combining the patches
def combine_patches(patches,shapes):
  
      h,w= shapes
      horizontal_image=[] 

      for i in range(h):
        image= np.hstack((patches[i*w],patches[i*w+1])) #combining first two patched images
        index= i*w+2 #As the first two patched imaged have been combined, now index selects third image

        for j in range(w-2): #-2 because the first the patches have been joined already
            image=np.hstack((image,patches[index+j]))
         
        horizontal_image.append(image) 
    
      actual_image=np.vstack((horizontal_image[0],horizontal_image[1]))
      for i in range(h-2):
        actual_image= np.vstack((actual_image,horizontal_image[2+i]))

      return actual_image    


def Total_combine_patches(patches,shapes,data_size,image_index=None):

  temporary= 0   
  total_actual_image=[]
  if image_index != None:
      image_index=image_index-1
      for i in range(image_index):
          hs= shapes[i][0]          
          ws= shapes[i][1]
          temporary= temporary+hs*ws
       
      h_,w_= shapes[image_index]
      return  combine_patches(patches[temporary:temporary+h_*w_],shapes[image_index])   


  for i in tqdm(range(data_size)):  
      h,w= shapes[i]
      image=combine_patches(patches[temporary:temporary+h*w],shapes[i])
      total_actual_image.append(image)
      temporary= h*w+temporary      

  return total_actual_image
    


In [ ]:
len(path_to_LR)

1993

In [ ]:
Total_patches(path_to_LR,48,'LR')

100%|██████████| 1993/1993 [1:00:09<00:00,  1.81s/it]


In [ ]:
/content/drive/MyDrive/SRGAN/LR

In [ ]:
a=os.listdir('/content/drive/My Drive/SRGAN/HR')
img= cv2.imread('/content/drive/My Drive/SRGAN/HR/5.png')
# cv2_imshow(img)
ar=np.empty(shape=(294,96,96,3))
for i in range(294):
    loc='drive/My Drive/SRGAN/HR' + a[i]
    print
    ar[i]= cv2.imread(loc)


In [ ]:
img= cv2.imread('drive/My Drive/SRGAN/HR/5.png')


In [ ]:
cv2_imshow(img)

AttributeError: ignored

In [ ]:
img= combine_patches(ar,shapes_HR[0])
cv2_imshow(img)

NameError: ignored

In [ ]:
shapes_HR[0]

[14, 21]

In [ ]:
shapes_HR[0]

[14, 21]

In [ ]:
len(os.listdir('/content/drive/My Drive/SRGAN/LR'))

591633

## Creating generator for data pipelining

In [ ]:
#parameter 
patch_size=48
batch_size=222

In [ ]:
def read(path_HR,path_LR):

    # 1)reading the content of the path

     # 3)Normalizing the image

    # 1)reading the content of the path
    content_HR=tf.io.read_file(path_HR)
    content_LR=tf.io.read_file(path_LR)
     
    # 2)decoding the content
    image_HR=tf.io.decode_png(content_HR,channels=3)
    image_LR=tf.io.decode_png(content_LR,channels=3)
     
    image_HR = tf.image.convert_image_dtype(image_HR, tf.float32)
    image_LR = tf.image.convert_image_dtype(image_LR, tf.float32)


    patches_HR= Total_patches(image_HR,patch_size,'HR')
    patches_LR= Total_patches(image_LR,patch_size,'LR')

    return patches_HR,patches_LR

In [ ]:
def pipeline(path_HR,path_LR):

   dataset=tf.data.Dataset.from_tensor_slices((path_HR,path_LR))
    
   
   #reading the images.....................................................
   dataset=dataset.map(read,num_parallel_calls=tf.data.experimental.AUTOTUNE)
   

   #Creating the batch
   dataset=dataset.batch(batch_size=batch_size,drop_remainder=True)
   
   #augmenting the batch images which will significantly improve the speed of transformation
   #dataset=dataset.map(augmentation,num_parallel_calls=tf.data.experimental.AUTOTUNE)


   #All the batches will be stored in the cache after the first iteration
   dataset.cache()

   #shuffeling 
   dataset=dataset.shuffle(buffer_size=25)
   dataset = dataset.repeat()

   dataset.prefetch(tf.data.experimental.AUTOTUNE)   


   return dataset



In [ ]:
def pipeline(HR_pat,LR_pat):

   dataset=tf.data.Dataset.from_tensor_slices((HR_pat,LR_pat))
    
   
   #reading the images.....................................................
   dataset=dataset.map(read,num_parallel_calls=tf.data.experimental.AUTOTUNE)

   #Creating the batch
   dataset=dataset.batch(batch_size=batch_size,drop_remainder=True)
   
   #augmenting the batch images which will significantly improve the speed of transformation
   #dataset=dataset.map(augmentation,num_parallel_calls=tf.data.experimental.AUTOTUNE)


   #All the batches will be stored in the cache after the first iteration
   dataset.cache()

   #shuffeling 
   dataset=dataset.shuffle(buffer_size=25)
   dataset = dataset.repeat()

   dataset.prefetch(tf.data.experimental.AUTOTUNE)   


   return dataset


In [ ]:
dataset= pipeline(path_to_HR[:100],path_to_LR[:100])
dataset


## GMGAN MODEL

In [ ]:
Growth_rate=16

def dense_block(inpt):
    """
    Dense block containes total 4 conv blocks with leakyRelu 
    activation, followed by post conv layer
    Params: tensorflow layer
    Returns: tensorflow layer
    """
    b1 = Conv2D(Growth_rate, kernel_size=3, strides=1, padding='same')(inpt)
    b1 = LeakyReLU(0.2)(b1)
    b1 = Concatenate()([inpt,b1])

    b2 = Conv2D(Growth_rate, kernel_size=3, strides=1, padding='same')(b1)
    b2 = LeakyReLU(0.2)(b2)
    b2 = Concatenate()([inpt,b1,b2]) 

    b3 = Conv2D(Growth_rate, kernel_size=3, strides=1, padding='same')(b2)
    b3 = LeakyReLU(0.2)(b3)
    b3 = Concatenate()([inpt,b1,b2,b3])

    b4 = Conv2D(Growth_rate, kernel_size=3, strides=1, padding='same')(b3)
    b4 = LeakyReLU(0.2)(b4)
    b4 = Concatenate()([inpt,b1,b2,b3,b4])

    b5 = Conv2D(Growth_rate, kernel_size=3, strides=1, padding='same')(b4)
    b5 = Lambda(lambda x:x*0.2)(b5)
    b5 = add([b5, inpt])
    
    return b5

def RRDB(inpt):
    """
    RRDB(residual in residual dense block) contained three dense  
    block, each block followed by beta contant multiplication(0.2) 
    and addition with dense block input layer.
    Params: tensorflow layer
    Returns: tensorflow layer
    """
    x = dense_block(inpt)

    x = dense_block(x)

    x = dense_block(x)

    x = Lambda(lambda x:x*0.2)(x)

    out = add([x,inpt])
    return out

In [ ]:
#Creating GMGAN

def Generator_model():
 input=Input(shape=(48,48,3),name='Input')

#First convolution layer
 x1= Conv2D(filters=Growth_rate,kernel_size=3,padding='Same',name='conv1')(input) 

 r1=RRDB(x1) #first RRDB

 r2=RRDB(r1) #second RRDB

 r3=RRDB(r2) #third RRDB

 r4=RRDB(r3) #forth RRDB

 r5=RRDB(r4) #fifth RRDB

#convolution layer after RRDB block
 x2= Conv2D(filters=Growth_rate,kernel_size=3,padding='Same')(r5) 

 x=add([x2,x1]) #skip connection

#Upsampling the LOW RESOLUTION IMAGE to increase the number of pixel by 2x in super resolution image
 x= UpSampling2D(size=(2,2))(x)

 x= Conv2D(filters=Growth_rate,kernel_size=3,padding='Same')(x) 

 output= Conv2D(filters=3,kernel_size=9,padding='Same')(x) 


 model=Model(input,output)
 model.summary()
 return model

generator=Generator_model()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 16)   448         Input[0][0]                      
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 16)   2320        conv1[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 48, 48, 16)   0           conv2d[0][0]                     
_______________________________________________________________________________________

In [ ]:
def conv_block(input,filter,kernel,stride):

        x=Conv2D(filters=filter,kernel_size=kernel,strides=stride,padding='valid')(input) 
        x=BatchNormalization()(x)
        x=LeakyReLU()(x)
        return x


In [ ]:
#Creating discriminator


input=Input(shape=(96,96,3),name='Input')


x= Conv2D(filters=Growth_rate,kernel_size=3,padding='Same',name='conv1')(input) 
x=LeakyReLU()(x)

x= conv_block(x,64,3,2)

x= conv_block(x,128,3,1)

x= conv_block(x,128,3,2)

x= conv_block(x,256,3,1)

x= conv_block(x,256,3,2)

x= conv_block(x,512,3,1)

x= conv_block(x,512,3,2)

x= conv_block(x,1,3,2)

x= GlobalAveragePooling2D()(x)
output= s()(x)
# x=Flatten()(x) #feature 
# x=Dense(1024)(x)
# x=LeakyReLU()(x)

# output=Dense(1,activation='sigmoid')(x)

discriminator=Model(input,output)
discriminator.summary()



## Losses

Generator loss:
   I_G= I_GA+I_Q+I_P+I_M 

   where:

   I_GA is adverserial loss

   I_Q is the GMSD loss

   I_p is the perceptual loss
   I_M is the mean square error 


In [ ]:
# @tf.function
def GMSD(HR,SR):

   # print('The shape of input image',HR.shape)

   sobel_HR=tf.image.sobel_edges(HR) #[H-2,W-2,3,2]
   # print('The  shape of sobel_HR:',sobel_HR.shape)
#    print('The output after sobel_HR',sobel_HR[0,0,0])

   sobel_HR= sobel_HR**2 #[H-2,2-2,3,2]
   G_HR=tf.math.sqrt(tf.math.reduce_sum(sobel_HR,axis=-1)) #[H-2,W-2,3,1] 
#    print('The output G_HR',G_HR[0,0,0])
   # print('The  shape of G_HR:',G_HR.shape)
#    print('\n')
#    print('*****************************************************')


   #calculating the gradient matrix value for super resolution image
 
   sobel_SR=tf.image.sobel_edges(SR) #[H-2,W-2,3,2]
   # print('The  shape of sobel_SR:',sobel_SR.shape)
#    print('The output after sobel_GR',sobel_SR[0,0,0])


   sobel_SR= sobel_SR**2 #[H-2,2-2,3,2]
   G_SR=tf.math.sqrt(tf.math.reduce_sum(sobel_SR,axis=-1)) #[H-2,W-2,3,1] 
#    print('The output after joining G_SR',G_HR[0,0,0])
   # print('The  shape of G_SR:',G_SR.shape)
#    print('\n')
#    print('******/***********************************************')


    #DEFINING THE PAREAMETER C
   c=tf.math.multiply(2*G_SR,G_HR)+0.000001
   d= tf.math.square(G_SR)+tf.math.square(G_HR)+0.000001
   GMS=c/d 
#    print('The  shape of GMS:',GMS)

   GMSM=tf.math.reduce_mean(GMS) #will get a scalar value
#    print('The GMSM value is :',GMSM)

   GMSD=tf.math.sqrt(tf.math.reduce_mean(tf.math.square((GMS-GMSM))))
#    print('The GMSD value is :',GMSD)
   return GMSD


def MSE(HR,SR):
    a= tf.keras.losses.MSE(SR,HR) #[batch,height,width]
    return tf.math.reduce_mean(a)
    

def perpectual_loss(HR,SR):
    VGG_19 = tf.keras.applications.VGG19(input_shape=(96,96,3),include_top=False,weights='imagenet')

    VGG_SR= VGG_19(SR)
    VGG_HR= VGG_19(HR)
    perpectual= MSE(VGG_SR,VGG_HR)
    return perpectual


In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# @tf.function
def generator_loss(HR,HR_pred,SR,SR_pred):
  
    I_GA= cross_entropy(tf.ones(shape=(batch_size,1)), SR_pred)
    # print('\n')
    # print('The value of GA IS ',I_GA)
    I_Q= GMSD(HR,SR)
    I_M= MSE(HR,SR)
    I_P= perpectual_loss(HR,SR)
    generator_los= 0.005*I_GA+I_Q+0.01*I_M+I_P
    return generator_los


In [ ]:
# @tf.function
def discriminator_loss(HR_pred,SR_pred):
    discriminator_los= 0.005*cross_entropy(tf.ones(shape=(batch_size,1)),HR_pred)+ cross_entropy(tf.zeros(shape=batch_size),SR_pred)
    return discriminator_los

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

## Training the model

In [ ]:
epochs=5

In [ ]:
checkpoint_dir = 'drive/My Drive/training_checkpoints'
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)


In [ ]:
@tf.function
def train(dataset, epochs):
  for epoch in range(epochs):
    #start = time.time()
    
    for data in tqdm(dataset):
        HR,LR= data

        with tf.GradientTape() as gen, tf.GradientTape() as dis:
    
  
            SR=generator(LR,training=True)
            SR_pred=discriminator(SR,training=True)
            HR_pred=discriminator(HR,training=True)
    
            gen_loss= generator_loss(HR,HR_pred,SR,SR_pred)
            # print('The gene loss',gen_loss)
  
            dis_loss= discriminator_loss(HR_pred,SR_pred)

            grad_generator= gen.gradient(gen_loss,generator.trainable_variables)
            grad_discriminator= dis.gradient(dis_loss,discriminator.trainable_variables)


            generator_optimizer.apply_gradients(zip(grad_generator, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(grad_discriminator, discriminator.trainable_variables))


    # Save the model at every epochs

        checkpoint.write(file_prefix = checkpoint_dir)

  # Generate after the final epoch



In [ ]:
train(dataset,epochs)





0it [00:00, ?it/s]



1it [00:05,  5.38s/it]



2it [00:11,  5.61s/it]



3it [00:16,  5.38s/it]



4it [00:21,  5.21s/it]



5it [00:25,  5.09s/it]



6it [00:31,  5.17s/it]



7it [00:36,  5.08s/it]



8it [00:40,  4.99s/it]



9it [00:45,  4.94s/it]



10it [00:50,  4.92s/it]



11it [00:56,  5.06s/it]



12it [01:00,  5.00s/it]



13it [01:05,  4.96s/it]



14it [01:10,  4.91s/it]



15it [01:15,  4.87s/it]



16it [01:20,  4.86s/it]



17it [01:25,  5.04s/it]



18it [01:30,  4.96s/it]



19it [01:35,  4.93s/it]



20it [01:40,  4.90s/it]



21it [01:44,  4.89s/it]



22it [01:49,  4.88s/it]



23it [01:54,  4.87s/it]



24it [02:00,  5.08s/it]



25it [02:05,  5.02s/it]



26it [02:09,  4.97s/it]



27it [02:14,  4.96s/it]



28it [02:19,  4.95s/it]



29it [02:24,  4.93s/it]



30it [02:29,  4.90s/it]



31it [02:34,  4.92s/it]



32it [02:39,  4.89s/it]



33it [02:44,  5.12s/it]



34it [02:49,  5.03s/it]



35it [02:54,  4.98s/it]



36it [02:59,  4.98s/it]



37it [03:04